In [1]:
import base64
import io
import json
import os
import time

from IPython.core.display import HTML
from IPython.display import Audio
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from models.config import PreprocessingConfig
from models.tts.delightful_tts.delightful_tts_refined import DelightfulTTS
from models.vocoder.univnet import UnivNet
from training.loss import Metrics
from training.preprocess import TacotronSTFT

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sample_rate = 22050

In [2]:
with open("training/datasets/speaker_id_mapping_libri.json") as f:
    id_mapping = json.load(f)
len(id_mapping)

2484

## Add the checkpoints to the `/checkpoints` folder and choose the appropriate version

In [3]:
# Checkpoints to use for demo
checkpoint_name = "logs_new_training_libri-360-swa_19_sec_epoch=190-step=70670"
checkpoint_univnet = "epoch=23-step=11616"


# checkpoint_path = f"./logs_100/{checkpoint_name}.ckpt"
# checkpoint_path = f"./logs_360_energy/{checkpoint_name}.ckpt"
# checkpoint_path = f"./logs/{checkpoint_name}.ckpt"
checkpoint_path = f"./checkpoints/{checkpoint_name}.ckpt"
checkpoint_path_univnet = f"./checkpoints/{checkpoint_univnet}.ckpt"

# Dataset url to use for demo
# dataset_url = "train-clean-100"
dataset_url = "train-clean-100"

# text_tts = """
# Casablanca: “But what about us?”
# “We’ll always have Paris.”
# The Wizard of Oz: “Lions? And Tigers? And Bears?”
# “Oh my!”
# Star Wars (A New Hope): “He’s almost in range.”
# “That’s no moon; it’s a space station.”
# Love Story: “Jenny, I’m sorry.”
# “Don’t. Love means never having to say you’re sorry.”
# No Country for Old Men: “Look, I need to know what I stand to win.”
# “Everything.”
# Forrest Gump: “I thought I’d try out my sea legs.”
# “But you ain’t got no legs, Lieutenant Dan.”
# Toy Story: “Buzz, you’re flying!”
# “This isn’t flying; this is falling with style.”
# As the snake shook its head, a deafening shout behind Harry made both of them jump.
# ‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
# """

text_tts = """
Casablanca: “But what about us?”
“We’ll always have Paris.”
The Wizard of Oz: “Lions? And Tigers? And Bears?”
“Oh my!”
"""

### Load from the checkpoint

In [4]:
model = DelightfulTTS.load_from_checkpoint(checkpoint_path, strict=False).to(device)
model.eval()

model.vocoder_module = UnivNet()
model.vocoder_module.eval()

print(f"Loaded checkpoint: {checkpoint_path}")

preprocess_config = PreprocessingConfig("english_only")
tacotronSTFT = TacotronSTFT(
    filter_length=preprocess_config.stft.filter_length,
    hop_length=preprocess_config.stft.hop_length,
    win_length=preprocess_config.stft.win_length,
    n_mel_channels=preprocess_config.stft.n_mel_channels,
    sampling_rate=preprocess_config.sampling_rate,
    mel_fmin=preprocess_config.stft.mel_fmin,
    mel_fmax=preprocess_config.stft.mel_fmax,
    center=False,
)
tacotronSTFT = tacotronSTFT.to(device)

/home/you/anaconda3/envs/tts_framework/lib/python3.11/site-packages/lightning/pytorch/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.2.0.post0, which is newer than your current Lightning version: v2.1.2


Loaded checkpoint: ./checkpoints/logs_new_training_libri-360-swa_19_sec_epoch=190-step=70670.ckpt


### Load the speakers list

In [5]:
from pydub import AudioSegment
import tempfile
import base64
import soundfile as sf

def plot_spectrogram(mel: np.ndarray):
    r"""Plots the mel spectrogram."""
    plt.figure(dpi=80, figsize=(10, 3))

    img = librosa.display.specshow(mel, x_axis="time", y_axis="mel", sr=sample_rate)
    plt.title("Spectrogram")
    plt.colorbar(img, format="%+2.0f dB")

    # Save the plot to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)

    # Convert the BytesIO object to a base64 string
    img_str = base64.b64encode(buf.read()).decode("utf-8")
    plt.close()

    return img_str

def gen_table(text_demo_with_metrics: list):
    # Initialize an empty string to store the HTML
    html = "<table border='1'>"

    html += f"<h4>TTS: </h4> {text_tts}"

    html += "<h4>Speakers: </h4>"
    html += """<tr>
        <th>SpeakerID</th>
        <th>Speaker Name</th>
        <th>Gender</th>
        <th>Subset</th>
        <th>Audio</th>
        <th>InfTime</th>
        <th>ermr</th>
        <th>jitter</th>
        <th>shimmer</th>
        <th>spec</th>
    </tr>"""

    for row in text_demo_with_metrics:
        # Speaker info
        speaker_id = row["speaker_id"]
        speaker_name = row["speaker_name"]
        gender = row["gender"]
        subset = row["subset"]
        execution_time = row["execution_time"]

        # Waveforms and mel spectrogram
        wav = row["wav"]
        mel = row["mel"]

        # Metrics
        ermr = row["ermr"]
        jitter = row["jitter"]
        shimmer = row["shimmer"]

        # Round the metrics to 3 decimal places
        metrics = [round(x, 3) for x in [ermr, jitter, shimmer]]

        # audio = Audio(wav, rate=sample_rate, autoplay=False)

        # Save the numpy array to a temporary wav file
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as wav_file:
            sf.write(wav_file.name, wav, sample_rate)

            # Convert wav to mp3
            audio = AudioSegment.from_wav(wav_file.name)
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=True) as mp3_file:
                audio.export(mp3_file.name, format="mp3")
                mp3_audio = base64.b64encode(mp3_file.read()).decode('utf-8')


        # Generate the spectrogram plot
        fig = plot_spectrogram(mel)
        # <td>{audio._repr_html_()}</td>

        # Add a row to the HTML table
        html += f"""<tr>
            <td>{speaker_id}</td>
            <td>{speaker_name}</td>
            <td>{gender}</td>
            <td>{subset}</td>
            <td><audio controls><source src="data:audio/mp3;base64,{mp3_audio}"></audio></td>
            <td>{execution_time}</td>
            <td>{metrics[0]}</td>
            <td>{metrics[1]}</td>
            <td>{metrics[2]}</td>
            <td><img src='data:image/png;base64,{fig}' /></td>
        </tr>"""

    # Close the HTML table
    html += "</table>"

    return HTML(html)


In [6]:
def gen_batch_table(batch: list, batch_idx: int):
    df = pd.DataFrame(batch)

    # Get the unique cluster labels
    df_ermr_sorted = df.sort_values(by="ermr")
    metrics = ["ermr", "jitter", "shimmer"]

    demo_dirname = f"logs/{dataset_url}_{checkpoint_name}"
    os.makedirs(demo_dirname, exist_ok=True)

    # Add header
    result = f"""
    <h3>Chunk #{batch_idx}: </h3>
    {df_ermr_sorted[["speaker_id", *metrics]].describe().to_html()}
    <h3>Chunk #{batch_idx} audio data: </h3>
    """

    # Generate and display the table for the chunk
    # for _, row in df_ermr_sorted.iterrows():
    result += gen_table(df_ermr_sorted.to_dict("records")).data # type: ignore

    # Save result as HTML
    with open(f"{demo_dirname}/chunk_{batch_idx}.html", "w") as f:
        f.write(result)


In [ ]:
# text_tts = """
# Casablanca: “But what about us?”
# “We’ll always have Paris.”
# The Wizard of Oz: “Lions? And Tigers? And Bears?”
# “Oh my!”
# Star Wars (A New Hope): “He’s almost in range.”
# “That’s no moon; it’s a space station.”
# Love Story: “Jenny, I’m sorry.”
# “Don’t. Love means never having to say you’re sorry.”
# No Country for Old Men: “Look, I need to know what I stand to win.”
# “Everything.”
# Forrest Gump: “I thought I’d try out my sea legs.”
# “But you ain’t got no legs, Lieutenant Dan.”
# Toy Story: “Buzz, you’re flying!”
# “This isn’t flying; this is falling with style.”
# As the snake shook its head, a deafening shout behind Harry made both of them jump.
# ‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
# """

text_tts = """
The Wizard of Oz: “Lions? And Tigers? And Bears?”
Toy Story: “Buzz, you’re flying!”
As the snake shook its head, a deafening shout behind Harry made both of them jump.
‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
"""

with open("training/datasets/speaker_id_mapping_libri.json") as f:
    id_mapping = json.load(f)

batch_size = 200
model.vocoder_module.to(device)

metrics = Metrics()
text_demo_with_metrics = []

speakers_df = pd.read_csv(
    "./datasets_cache/LIBRITTS/LibriTTS/speakers.tsv",
    sep="\t",
    names=["READER", "GENDER", "SUBSET", "NAME"],
)

subsets = ["train-clean-100", "train-clean-360", "train-other-500"]

speakers = speakers_df[speakers_df['SUBSET'].isin(subsets)]

start_chunk = 11  # The chunk to start from
start_idx = start_chunk * batch_size  # The index to start from

for idx, speaker in enumerate(speakers.to_dict("records")):
    # Skip the speakers that have already been processed
    if idx <= start_idx:
        continue

    speaker_id = speaker["READER"]
    speaker_id = id_mapping.get(str(speaker_id))
    speaker_name = speaker["NAME"]
    gender = speaker["GENDER"]
    subset = speaker["SUBSET"]

    with torch.no_grad():
        speaker_id_ = torch.tensor([int(speaker_id)], device=device)
        start_time = time.time()
        wav = model.forward(text_tts, speaker_id_)
        end_time = time.time()
        (
            ermr,
            jitter,
            shimmer,
        ) = metrics.wav_metrics(wav.unsqueeze(0))
        execution_time = round(end_time - start_time, 2)

        mel = tacotronSTFT.get_mel_from_wav(wav)

        text_demo_with_metrics.append({
            "speaker_id": speaker_id,
            "speaker_name": speaker_name,
            "gender": gender,
            "subset": subset,
            "wav": wav.detach().cpu().numpy(),
            "execution_time": execution_time,
            "mel": mel.detach().cpu().numpy(),
            "ermr": ermr,
            "jitter": jitter,
            "shimmer": shimmer,
        })

    if idx > 0 and idx % batch_size == 0:
        gen_batch_table(text_demo_with_metrics, idx // batch_size)
        text_demo_with_metrics = []

    torch.cuda.empty_cache()

gen_batch_table(text_demo_with_metrics, 999)


In [7]:
# chunk_1
speaker_ids1 = [
    32, # US, nan 	M 	train-clean-100, min intonation, calm, timbre
    130, # BR, 	Peter of Buckinghamshire England 	M 	train-other-500, timbre, min intonation, calm, slow
    164, # US, 	Ed Good 	M 	train-other-500, timbre, intonation, warm and calm, good voice
    176, # US, 	Micah Sheppard 	M 	train-clean-360, intonation, calm
    68, # US,	Alex Buie 	M 	train-clean-100,  min intonation, energized
    15, # US, 	Chip 	M 	train-other-500, intonation, interesting voice
    145, # US, 	Mr. Baby Man 	M 	train-clean-360, timbre, intonation, interesting voice
    196, # US, 	Mike Kauffmann 	M 	train-clean-360,  min intonation, average calm
    160, # BR,	deadwhitemales 	M 	train-clean-100, intonation, energized, interesting voice
    108, # US, 	Kevin McAsh 	M 	train-clean-360, min intonation, energized
    99, # US, 	Stewart Wills 	M 	train-clean-100, min intonation, avg energy
    50, # US, 	Dan Threetrees 	M 	train-clean-360, min intonation, avg energy
    76, # US, 	ML Cohen 	M 	train-other-500, intonation, energy, timbre
    142, # US, 	Michael Sirois 	M 	train-other-500, intonation, energy
    95, # US, 	Vinny Bove 	M 	train-clean-360, intonation, energy, timbre
    169, # US, 	Richard Grove 	M 	train-other-500, intonation, energy
    147, # US, 	Eileen George 	F 	train-clean-360, intonation, calm
    92, # BR 	vlooi 	F 	train-other-500 teenager voice, intonation, energized
    117, # US, 	Caitlin Kelly 	F 	train-clean-360, intonation, energized, interesting voice
    89, # US, 	Paula Berinstein 	F 	train-clean-360, min intonation, energized
    182, # US 	Katy Preston 	F 	train-other-500, intonation, energized
    105, # US 	Marian Brown 	F 	train-clean-360, min intonation, energized, timbre, quality GOOD
    11, # BR 	Linton 	F 	train-other-500, min intonation, calm
    18, # US 	Sherry Crowther 	F 	train-clean-100, intonation, energized, timbre
    38, # US 	Kurt Copeland 	M 	train-clean-360, intonation, energized
    52, # US, 	Cori Samuel 	F 	train-other-500, intonation, energized, timbre
    21, # US, 	Kelly Bescherer 	F 	train-other-500, intonation, energized, timbre
    48, # US, 	Rosalind Wills 	F 	train-clean-100, intonation, timbre, poor quality, but interesting voice
    63, # US, 	Linda Wilcox 	F 	train-other-500, very poor quality, but VERY interesting intonation
    45, # US, 	Catharine Eastman 	F 	train-clean-100, very quality, interesting intonation
    193, # US, 	Nomenphile 	F 	train-other-500, avg quality, intonation
    207, # US 	Sage Tyrtle 	F 	train-other-500, avg quality, interesting intonation and timbre
    73, # US 	Claire Goget 	F 	train-clean-100, avg quality, intonation
    42, # US 	Jennifer Crispin 	F 	train-clean-360, quality, calm
    113, # US 	Alice Elizabeth Still 	F 	train-other-500, avg quality, intonation
    49, # US 	Kristen McQuillin 	F 	train-clean-100, avg quality, interesting intonation and timbre
    185, # US	Kim Braun 	F 	train-clean-360, avg quality, avg intonation, but interesting timbre
    165, # US 	Elisabeth Shields 	F 	train-clean-100, avg quality, interesting intonation, timbre
    36, # US 	chriss the girl 	F 	train-other-500, interesting intonation, timbre
    96, # US 	Kymm Zuckert 	F 	train-other-500, avg quality, interesting intonation, timbre of middle age wooman
    102, # US 	Maureen S. O'Brien 	F 	train-clean-100, avg quality, interesting intonation
    64, # US 	Christiane Levesque 	F 	train-clean-360, young voice, calm
    30, # BR, 	Ophelia Darcy 	F 	train-other-500, avg quality, intonation
    0, # US 	Kristin LeMoine 	F 	train-clean-360, avg quality, intonation, fast-speaker
    192, # US, 	Nocturna 	F 	train-clean-100, avg quality, intonation
    146, # US 	Jim Cadwell 	M 	train-other-500, avg quality, intonation
    80, # US	Fox in the Stars 	F 	train-clean-100, avg quality, intonation
    162, # BR	Mike Gardom 	M 	train-other-500, avg quality, intonation, timbre
    66, # US 	Maddie 	F 	train-clean-360, intonation, timbre
    124, # US 	Steve Karafit 	M 	train-clean-100, poor quality, intonation, timbre, COOL
    123, # US 	Sean McGaughey 	M 	train-clean-360, poor quality, intonation, timbre
    55, # US, 	Patricia Oakley 	F 	train-clean-360, intonation, timbre
    91, # BR 	Chris Goringe 	M 	train-other-500, intonation, timbre
    171, # US	Paul Harvey 	M 	train-clean-360, avg quality, intonation, timbre
    61, # US	John Greenman 	M 	train-other-500, intonation, timbre
    122,  # US  carnright 	M 	train-clean-100, avg intonation, timbre
    82, # BR 	Andy Minter 	M 	train-other-500, intonation, GOOD timbre,
    118, # US 	Brenda Dayne 	F 	train-clean-360, timbre, calm
    106, # US	Mark F. Smith 	M 	train-clean-360, GOOD timbre, interesting intonation
    187, # US	Lenny Glionna Jr. 	M 	train-other-500, intonation, timbre
    158, # US 	Randy Phillips 	M 	train-clean-100, avg intonation, timbre
    83, # US	Graham Williams 	M 	train-other-500, intonation, timbre
    181, # BR 	Jon Ingram 	M 	train-other-500, intonation, timbre
    8, # US	Denny Sayers 	M 	train-clean-100, timbre
    205, # US	Robert Garrison 	M 	train-clean-360, timbre, interesting intonation
    172, # US 	Harvey Chinn 	M 	train-other-500, great timbre, intonation
]
# chunk_2
speaker_ids2 = [
    305, # US 	Michael Crowl 	M 	train-clean-360, timbre, intonation
    417, # US	Kiki Baessell 	F 	train-clean-360, timbre, intonation
    241, # US	Jean O'Sullivan 	F 	train-clean-360, timbre, intonation
    412, # US	Nichole Karl 	F 	train-other-500, timbre
    294, # US	Diana Kiesners 	F 	train-clean-360, timbre, intonation
    414, # US 	Christabel 	F 	train-clean-100, great and calm timbre
    251, # US	Aaron Benedict 	M 	train-other-500, timbre, intonation
    218, # US	Joy Chan 	F 	train-other-500, timbre, intonation, quality
    259, # BR	miette 	F 	train-other-500, timbre, intonatio
    275, # US	Zale Schafer (Rose May Chamberlin Memorial Foundat 	F 	train-clean-360, timbre, intonation
    368, # US	Susie G. 	F 	train-other-500, teenage voice, timbre, intonation
    327, # BR	mjd-s 	F 	train-other-500, timbre, intonation
    366, # US 	pattymarie 	F 	train-clean-360, calm, timbre
    399, # US	entada 	F 	train-clean-360, timbre, intonation
    273, # BR 	Andrew Lebrun 	M 	train-other-500, timbre, intonation
    352, # US 	Mur Lafferty 	F 	train-clean-360, timbre, enery, young voice
    283, # US	Bethany Simpson 	F 	train-clean-360, timbre, intonation
    248, # BR 	fieldsofgold 	M 	train-other-500, timbre, intonation
    372, # US	Jim Mullins 	M 	train-clean-360, timbre, intonation, quality
    250, # US	Quentin 	M 	train-clean-360, unique timbre, intonation, quality
    403, # US	Igor Teaforay 	F 	train-clean-360, timbre, avg intonation
    375, # US	iscatel 	M 	train-clean-360, unique timbre, avg intonation
    355, # US 	Lana Taylor 	F 	train-clean-100, unique timbre, intonation
    220, # US	Tina Tilney 	F 	train-clean-360, unique timbre, high intonation
    392, # US 	Brooks Jensen 	M 	train-clean-360, timbre, avg speed, avg intonation
    255, # US 	Caroline Mercier 	F 	train-clean-360, timbre, avg speed, intonation
    292, # US 	Tamara R. Schwartz 	F 	train-clean-100, timbre, avg speed, intonation
    258, # US 	Russ Maxwell 	M 	train-clean-360, timbre, intonation, unique voice
    233, # US	mikenkat 	M 	train-clean-360, timbre, intonation
    394, # US 	swroot 	F 	train-clean-360, timbre, intonation, unique voice, good quality
    382, # US	Kelli Robinson 	F 	train-clean-360, timbre, intonation, avg quality
    322, # US	Mitchell Dwyer 	M 	train-clean-360, timbre, intonation, avg quality, unique voice
    285, # US	koijmonop 	M 	train-clean-360, timbre, intonation, avg quality, unique voice
    290, # US	J. M. Smallheer 	F 	train-clean-360, timbre, intonation, avg quality
    383, # US	Mike Roop 	M 	train-other-500, timbre, intonation, quality, unique voice
    263, # US 	Eric S. Piotrowski 	M 	train-clean-360, timbre, intonation, avg quality
    373, # BR	Brooks Seveer 	M 	train-clean-360, timbre, intonation, avg quality, unique voice
    340, # US	Nick Gallant 	M 	train-clean-100, unique timbre
    331, # BR 	Luke Venediger 	M 	train-other-500, timbre, intonation
    234, # BR	Menno 	M 	train-other-500, timbre, intonation, unique voice
    398, # BR 	Sam Fold 	M 	train-other-500, timbre, intonation
    254, # BR	Rebecca Dittman 	M 	train-other-500, timbre, intonation
    264, # US	KentF 	M 	train-clean-360, timbre, intonation
    214, # US 	Scott Splavec 	M 	train-clean-100, timbre, intonation
    359, # BR	Greg Bryant 	M 	train-clean-100, timbre, intonation
    253, # US	Frank 	M 	train-other-500, timbre, intonation
    249, # US	Bill Stackpole 	M 	train-clean-360, timbre, intonation
    236, # US	Matthew Shepherd 	M 	train-clean-360, timbre, intonation, unique voice
    390, # US	JimmyLogan 	M 	train-clean-360, avg intonation, timbre
    216, # US	Dave Ranson 	M 	train-clean-100, unique voice, timbre, intonation
    215, # US	Mark Bradford 	M 	train-clean-360, timbre, intonation
    212, # US	Glen Hallstrom 	M 	train-other-500, timbre, intonation
    314, # US	Carl Vonnoh, III 	M 	train-clean-360, unique voice, timbre, intonation
    347, # US	Anadaxis_Canejia 	M 	train-other-500, unique voice, timbre, intonation, good quality
    330, # US	Aaron Andradne 	M 	train-clean-360, timbre, intonation
    393, # US	Tim Lundeen 	M 	train-clean-360, timbre, intonation
    40,  # BR 	Justin Brett 	M 	train-other-500, unique voice, timbre, intonation, good quality
    386, # US 	Michael Kirkpatrick 	M 	train-clean-360, unique intonation, timbre
    315, # US	Jean Crevier 	M 	train-other-500, unique intonation, timbre
    349, # US	brenthumphries 	M 	train-other-500, unique timbre, min intonation
    242, # US	J. Hall 	M 	train-other-500, unique timbre, intonation
    308, # US	Eric Connover 	M 	train-other-500, timbre, intonation
    313, # BR	Tim Bulkeley 	M 	train-other-500, timbre, intonation
    334, # US	mawrtea 	M 	train-clean-360, unique voice, timbre, intonation
]
# chunk_3
speaker_ids3 = [
    469, # US 	Christian Pecaut 	M 	train-clean-360, clear voice, min intonation
    601, # BR	Jonathan Horniblow 	M 	train-other-500, clear voice, min intonation
    482, # US	Estragon 	M 	train-clean-360, clear voice, min intonation
    580, # US	Kyle M. 	M 	train-clean-360, clear voice, min intonation, timbre
    463, # US	Chris Hughes 	M 	train-other-500, clear voice, min intonation, timbre, energized
    472, # BR	Tim Makarios 	M 	train-other-500, min intonation
    536, # US	Robert Flach 	M 	train-other-500, intonation, timbre, unique voice
    569, # US 	Arouet 	M 	train-clean-360, avg intonation, timbre
    523, # US	Michael Loftus 	M 	train-clean-360, avg intonation, timbre
    454, # US	Tim Gregory 	M 	train-clean-100, avg intonation
    425, # US	RedToby 	M 	train-clean-360, avg intonation, timbre, unique voice
    574, # US	Daniel Shorten 	M 	train-clean-100, avg intonation, timbre, clear voice
    465, # US 	Leonie Rose 	F 	train-clean-100, avg intonation, timbre
    481, # US 	Scott Sherris 	M 	train-clean-360, intonation, timbre
    531, # US 	Fr. Richard Zeile of Detroit 	M 	train-other-500, intonation, timbre, unique voice
    628, # US	Bryan Ness 	M 	train-clean-100, intonation, timbre, unique voice
    427, # US 	John Lieder 	M 	train-clean-360, intonation, timbre
    527, # US	Jason Isbell 	M or F 	train-clean-360, intonation, timbre
    468, # US	Jenilee 	F 	train-other-500, energized, young voice, intonation
    441, # US	roolynninms 	F 	train-clean-100, energized, intonation
    430, # BR 	Millbeach 	F 	train-other-500, intonation, timbre
    498, # US	Chris Gladis 	M 	train-clean-100, intonation, timbre
    435, # US 	Debra Lynn 	F 	train-other-500, intonation, timbre
    499, # US 	Tammy Sanders 	F 	train-clean-100, intonation, timbre, unique voice
    429, # US 	Giles Baker 	M 	train-other-500, timbre
    495, # US	Janet Friday 	F 	train-clean-360, INTONATION, timbre
    456, # US	Catherine Fitz 	F 	train-clean-360, timbre
    617, # US 	PJ 	F 	train-other-500, timbre
    445, # US	Jennette Selig 	F 	train-clean-360, intonation, timbre
    552, # US	Mim Ritty 	F 	train-clean-100, intonation, timbre
    561, # US	Lorelle Anderson 	F 	train-clean-100, intonation, timbre, quality
    447, # US	Lee Ann Howlett 	F 	train-clean-360, intonation, timbre, quality
    604, # BR	Anne-Marie 	F 	train-other-500, intonation, timbre
    450, # US	Heather Duncan 	F 	train-clean-360, intonation, unique voice
    502, # US	Lee Elliott 	F 	train-other-500, intonation
    517, # US	Madame Tusk 	F 	train-other-500, intonation
    605, # BR	Classicsfan 	F 	train-other-500, intonation, unique voice, timbre
    426, # US	Megan Stemm-Wade 	F 	train-clean-100, clear voice, min intonation
    544, # US	Miranda Stinson 	F 	train-clean-360, unique voice
    568, # US	Michael Yourshaw 	M 	train-other-500, clear voice
    588, # US	Kalynda 	F 	train-clean-360, clear voice, timbre
    477, # US	Patti Brugman 	F 	train-other-500, clear voice
    607, # BR 	Steph 	F 	train-other-500, clear voice
    479, # US 	Wina Hathaway 	F 	train-other-500, intonation, timbre
    618, # US	Linnea 	F 	train-other-500, timbre
    549, # US	AmyAG 	F 	train-other-500, teenager voice
    596, # US	Jo 	F 	train-other-500, timbre, min intonation
]
# chunk 4
speaker_ids4 = [
    770, # US	Pete Williams, Pittsburgh, PA 	M 	train-clean-360, clear, calm voice
    730, # US	Nick Gisburne 	M 	train-other-500, clear, calm voice
    680, # US	Kevin Kivikko 	M 	train-clean-360, min intonation, calm voice, timbre
    791, # US	David Kleparek 	M 	train-clean-100, clear, calm voice
    800, # US	Jack Farrell 	M 	train-clean-360, clear, calm voice, good low tone
    839, # BR	Ben Dutton 	M 	train-other-500, clear, calm voice, min intonation
    661, # US	James Gladwin 	M 	train-other-500, calm voice, min intonation
    648, # US	Muhammad Mussnoon 	M 	train-other-500, timbre
    798, # US	Wyatt 	M 	train-other-50, calm voice, timbre, avg intonation
    699, # US	Michael Macedonia 	M 	train-clean-360, calm voice, timbre
    757, # US	Roger Melin 	M 	train-clean-360, timbre, avg intonation, unique voice
    782, # US	Alec Daitsman 	M 	train-clean-360, timbre, avg intonation
    816, # US	Michael Bradford 	M 	train-clean-360, timbre, avg intonation, teenager voice
    846, # US	Gayland Darnell 	M 	train-clean-360, senior voice, timbre, avg intonation
    831, # US	David Federman 	M 	train-other-500, timbre, avg intonation
    817, # US	texttalker 	M 	train-clean-360, timbre, avg intonation
    819, # US	n8evv 	M 	train-clean-360, timbre, intonation
    672, # BR	Stuart Bell 	M 	train-other-500, timbre, intonation, clear voice
    837, # US	johnb 	M 	train-other-500, timbre, intonation
    775, # US	Ralph Volpi 	M 	train-clean-360, senior voice, timbre, intonation
    685, # US	Nikki Sullivan 	F 	train-clean-100, clear voice, avg intonation, timbre
    666, # US	Kelly Dougherty 	M 	train-clean-360, clear voice, avg intonation, timbre
    809, # US	Anna-Maria Viola 	F 	train-clean-360, timbre, avg intonation
    640, # US	David A. Stokely 	M 	train-other-500, emotional, timbre, intonation, unique voice
    818, # US	Matt Warzel 	F 	train-clean-360, timbre, intonation
    711, # US	Julie Bynum 	F 	train-clean-360, senior voice, timbre, min intonation
    766, # US	Clive Catterall 	M 	train-other-500, avg intonation, timbre
    649, # US	Scarlett! 	F 	train-clean-360, intonation, timbre
    732, # US	Tysto 	M/F	train-clean-360, intonation, timbre
    769, # US, Alan Brown 	M 	train-other-500, avg intonation, timbre
    677, # US	Allyson Hester 	F 	train-other-500, intonation, timbre, unique voice
    833, # US	Alana Jordan 	F 	train-clean-360, intonation, timbre
    676, # US	Jennifer 	F 	train-clean-100, intonation, timbre
    710, # US	Sheila Morton 	F 	train-clean-100, intonation, timbre, unique voice
    645, # US	Micah 	F 	train-other-500, intonation, timbre, unique voice, teenager voice
    771, # US	Isosceles 	F 	train-clean-360, intonation, timbre
    636, # US	Matthew Howell 	F 	train-other-500, timbre, avg intonation
    822, # US	kristiface 	F 	train-clean-360, timbre, intonation, unique voice
    830, # US	musici123 	F 	train-other-500, timbre, intonation, unique voice
    657, # US	Shannon 	F 	train-other-500, timbre, intonation
    797, # US	Chris Jones 	F 	train-other-500, timbre, intonation, calm voice
    835, # BR	Rachel Lintern 	F 	train-other-500, timbre, intonation
    761, # US	Susan Umpleby 	F 	train-clean-100, timbre, intonation, unique voice
    653, # US	cucciasv 	F 	train-other-500, timbre, avg intonation
    751, # US	Ralph Snelson 	M 	train-other-500, timbre, avg intonation, unique voice
    848, # BR	senshisteph 	F 	train-other-500, timbre, intonation
    808, # US	M. J. Boyle 	F 	train-other-500, timbre, intonation, senior voice
    840, # US	B. Grebe 	F 	train-clean-360, timbre, intonation, unique voice
    742, # US	Katherine Holt 	F 	train-other-500, timbre, intonation
    834, # US	Serin 	F 	train-other-500, unique voice
    668, # US	Jan Baxter 	F 	train-clean-360, intoantion, timbre, calm voice
    752, # US	Cat Schirf 	F 	train-other-500, intoantion, timbre
    641, # US	Eliza Horne 	F 	train-other-500, intoantion, timbre
    644, # US	Cynthia Zocca 	F 	train-clean-360, intoantion, timbre
    781, # US	Megan Kunkel 	F 	train-other-500, intoantion, timbre, teenager voice
    727, # US	Jodi Krangle 	F 	train-clean-360, timbre
    719, # US	Charlene V. Smith 	F 	train-other-500, intoantion, timbre
    804, # BR	FirstKnight 	F 	train-other-500, unique voice, intoantion, timbre
    675, # US	inkwelldragon 	F 	train-clean-360, intoantion, timbre
    697, # US	Jennie Hughes 	F 	train-other-500, timbre
    731, # IND	Priya, India 	F 	train-other-500, avg intoantion, timbre
    741, # US 	Nick Marsh 	M 	train-other-500, intoantion, timbre, old man voice, unique voice
]
# chunk 5
speaker_ids5 = [
    922,  # ricell
    1000,  # artos
    1007,  # Mike Conrad
    858,  # Scott Merrill
    943,  # Matthew C. Heckel
    984,  # woggy298
    936,  # BUAES
    935,  # Topaz
    977,  # Logan McCamon
    946,  # Cantor
    1030,  # Ancient mariner
    1046,  # Preston McConkie
    1022,  # peac
    908,  # Quentin Manuel
    924,  # Andrew Coleman
    964,  # Utek
    950,  # davechase
    1020,  # nihilist00
    1043,  # B. G. Oxford
    881,  # mpetranech
    852,  # Steven Proctor
    995,  # Parrot
    1045,  # joi
    1048,  # tornadogrrrl
    900,  # peaceuntoyou
    932,  # Raerity
    1005,  # Beatrice
    851,  # Jennifer Lott
    897,  # Jan Dawn Doronila
    1041,  # mjbrichant
    863,  # K Hindall
    937,  # Sarah Gutierrez
    1049,  # Diana Solomon
    1001,  # TriciaG
    934,  # Darla
    947,  # Larissa Little
    944,  # Sarafina Suransky
    870,  # Barbara Bulkeley
    923,  # Jane Greensmith
    1047,  # Hannah Dowell
    967,  # Stephanie Land
    929,  # Petra
    963,  # MichelleHarris
    891,  # anoldfashiongirl
    890,  # PopularOutcast
    992,  # Fran
]
# chunk_6
speaker_ids6 = [
    1143, #	Keith Henige 	M 	train-other-500
    1159, #	Matt Wills 	M 	train-clean-360
    1127, #	C.J. Casey 	M 	train-other-500
    1235, #	Richard Kilmer 	M 	train-other-500
    1092, #	BenW 	M 	train-other-500
    1270, # Brendan Tannam 	M 	train-other-500
    1214, #	David Baldwin 	M 	train-clean-360
    1255, #	Daniel Paashaus 	M 	train-other-500
    1152, #	Brian Keith Barnes 	M 	train-clean-360
    1158, #	StarrDog 	M 	train-other-500
    1256, #	Graeme Dunlop 	M 	train-other-500
    1215, #	Kevin Maxson 	M 	train-clean-360
    1274, #	Jud Niven 	M 	train-clean-360
    1168, #	Epistomolus 	M 	train-clean-100
    1089, #	Bill Ruhsam 	M 	train-clean-360
    1142, #	Jonathan Burchard 	M 	train-other-500
    1090, #	Termin Dyan 	M 	train-other-500
    1109, #	Martin Geeson 	M 	train-other-500
    1230, #	Troy Bond 	M 	train-other-500
    1150, #	TexasSteve 	M 	train-other-500
    1191, # Denise Lacey 	F 	train-other-500
    1259, # 	Megan Argo 	F 	train-other-500
    1238, #	madmouth 	F 	train-other-500
    1135, #	Linda Andrus 	F 	train-clean-360
    1247, #	Sarah LuAnn 	F 	train-clean-100
    1115, #	Rachel Gatwood 	F 	train-clean-360
    1065, #	Bob Sherman 	M 	train-other-500
    1204, #	Dale A. Bade 	F 	train-clean-360
    1174, #	Frances Marcinkiewicz 	F 	train-
    1257, #	Availle 	F 	train-other-500
    1239, #	Rachell Lovett 	F 	train-clean-360
    1273, #	gmiteva 	F 	train-other-500
    1242, #	Richard Ellwood 	M 	train-clean-360
    1093, #	Katie Riley 	F 	train-clean-360
    1063, # 	SuD 	F 	train-other-500
    1098, #	Kerry Hiles 	F 	train-other-500
    1254, #	Rosie 	F 	train-clean-100
    1157, # 	Bev J Stevens 	F 	train-clean-360
    1184, #	Joseph Couves 	F 	train-other-500
    1253, #	Caroline Shapiro 	F 	train-other-500
    1183, #	Evelyn Clarke 	F 	train-other-500
    1082, #	Symmie 	F 	train-clean-360
    1128, #	Linda Ferguson 	F 	train-other-500
    1108, #	Paul McCartan 	M 	train-other-500
    1202, #	Joy Easton 	F 	train-clean-360
    1226, #	serenitylee 	F 	train-clean-360
    1105, #	Bridget Gaige 	F 	train-clean-360
    1229, #	RoseA 	F 	train-clean-360
    1181, #	J. Rebecca Franklin 	F 	train-clean-360
    1231, #	Abigail Bartels 	F 	train-other-500
    1182, #	tabithat 	F 	train-other-500
    1217, #	JimOCR 	M 	train-other-500
    1171, #	Roberta Carlisle 	F 	train-other-500
    1268, #	A. Janelle Risa 	F 	train-clean-100
    1243, #	Rachel P. 	F 	train-clean-360
    1071, #	js392 	F 	train-other-500
]
# chunk_7
speaker_ids7 = [
    1428,  # Gary Dzierlenga
    1315,  # John Dennison
    1376,  # mevans
    1330,  # William Peck
    1400,  # scrawl
    1314,  # Michael Wolf
    1425,  # Jonah Cummings
    1438,  # Tom Barron
    1281,  # garbageman99
    1414,  # Preston Scrape
    1375,  # Frank Adams
    1410,  # Zachary Johnson
    1365,  # Eric Leach
    1302,  # davidb
    1354,  # Kristen Zaza
    1346,  # Jeanie
    1320,  # Anita Fleming
    1370,  # Savanna Herrold
    1290,  # Veronica Jenkins
    1437,  # Charles RUHE
    1297,  # May Low
    1440,  # P Moscato
    1433,  # browneyedgirl32382
    1366,  # cher0520
    1285,  # Ric F
    1399,  # Jeanne Luft
    1402,  # Angel5
    1303,  # kiwafruit
    1301,  # Barbara Clements
    1453,  # Anna-Lisa Ott
    1374,  # pachayes
    1373,  # Maria Therese
]
# chunk 8
speaker_ids8 = [
    1649,  # Phineas Redux
    1691,  # sparks0314
    1672,  # Mike Wajda
    1539,  # Nathan Jordan
    1610,  # jgoffena
    1512,  # Matt Soar
    1526,  # Mike Harris
    1647,  # Patrick Reinhart
    1636,  # jessecoy
    1676,  # Gargoyle
    1595,  # Matthew Reece
    1609,  # Jacob Paul Starr
    1671,  # bobbybrill
    1555,  # Andrew Nelson
    1657,  # alwpoe
    1592,  # jerryB
    1505,  # Rom Maczka
    1565,  # bryan.peterson
    1644,  # Christopher Maust
    1695,  # Tina Nuzzi
    1702,  # Sirmelja
    1697,  # Lonelle Yoder
    1596,  # Joyce Couch
    1660,  # Jerry Romero
    1524,  # Elizabeth Barr
    1643,  # Linette Geisel
    1543,  # Lauren McCullough
    1613,  # Elsa Youngsteadt
    1662,  # GabrielleC
    1587,  # Claudia Wilson
    1641,  # Kirsten Wever
    1614,  # Jennifer Dionne
    1603,  # Christine Rodriguez
    1546,  # Carrie Heyes
    1579,  # Linda Velwest
    1638,  # Laura Victoria
    1651,  # Debbie Pieterse
    1554,  # Natalie Sullivan
    1656,  # Sharon Omi
    1607,  # Lynda Sizemore
    1670,  # dmbrought
    1659,  # kelleywyskiel
]
# chunk_9
speaker_ids9 = [
    1785, # BR	TRUEBRIT 	M 	train-other-500, timbre, min intonation
    1776, # US	dan_h 	M 	train-other-500, avg intoantion, timbre
    1791, # US	David Cummings 	M 	train-clean-360, avg intoantion, timbre
    1707, # US 	David Olson 	M 	train-other-500, unique voice, intoantion, timbre
    1801, # US	John O 	M 	train-other-500, unique voice, min intoantion, timbre
    1903, # US	stephenreader 	M 	train-other-500, velvet unique voice
    1856, # BR	Craig Gulliver 	M 	train-other-500, avg intoantion, timbre
    1906, # US	Tony Russell 	M 	train-clean-360, dictor voice, avg intoantion, timbre
    1806, # US	Marc Pizzuti 	M 	train-other-500, avg intoantion
    1828, # US	David Wales 	M 	train-clean-100, avg intoantion, timbre
    1849, # US	Fred DeBerardinis 	M 	train-clean-100, avg intoantion, timbre, unique voice
    1712, # BR	Steve Belleguelle 	M 	train-other-500, avg intoantion, timbre
    1853, # US	Ron Altman 	M 	train-other-500, intoantion, unique voice, timbre
    1805, # US	Vince Dee 	M 	train-clean-100, unique voice, timbre
    1838, # US	Morey Kunin 	M 	train-clean-360, young voice, intoantion, timbre
    1862, # US Alexandre Laplante 	M 	train-clean-360, young voice, intoantion, timbre
    1915, # US	T.E. McHenry 	M 	train-other-500, unique voice, dictor voice, intoantion, timbre
    1772, # US	haggisreflux 	M 	train-clean-360, intoantion, timbre
    1831, # BR	Nigel Boydell 	M 	train-other-500, unique voice, intoantion, timbre
    1706, # BR	Deborah Knight 	F 	train-clean-100, unique voice, intoantion, timbre
    1908, # US	fshort 	F 	train-other-500
    1783, # US	Sarah Crampton 	F 	train-clean-360
    1781, # US	humanode 	M 	train-clean-360, unique voice
    1779, # US	Wendy Almeida 	F 	train-clean-360
    1839, # US	James E. Carson 	M 	train-clean-360
    1724, # US	Juliana M. 	F 	train-clean-360
    1728, # US	Vinnie Tesla 	M 	train-clean-360
    1907, # US	Snapdragon 	F 	train-other-500
    1881, # US	Julienne 	F 	train-other-500
    1802, # US	selway 	F 	train-other-500
    1826, # US	John Hoerr 	M 	train-clean-100
    1725, # BR	Ruth Kidson 	F 	train-other-500
    1764, # US	ReadWriteLib 	F 	train-clean-360
    1794, # US	Michelle Remington 	F 	train-clean-360
    1880, # US	Christine Nendza 	F 	train-clean-360
    1848, # US	Monica Knuppe 	F 	train-clean-360
    1736, # US	Spike Holcomb 	F 	train-other-500
    1841, # US	Elena 	F 	train-clean-360
    1836, # US	Kendall Ashyby 	F 	train-other-500
    1741, # US	anjieliu 	F 	train-other-500
    1803, # US	Susan Hanfield 	F 	train-clean-360
    1761, # US	EliMarieHK 	F 	train-other-500
    1745, # US	Janet 	F 	train-clean-360
    1713, # US	dobsonfly 	F 	train-clean-100
    1716, # US	EyeBones 	F 	train-clean-360
    1814, # US	polkadotish 	F 	train-other-500
    1709, # US	CrowGirl 	F 	train-other-500
    1763, # US	Gen Jones 	F 	train-clean-360
    1808, # US 	Rebecca King 	F 	train-clean-360
    1811, # US	Michelle Day 	F 	train-clean-360
    1857, # US	Amanda Friday 	F 	train-clean-360
    1893, # US	KirksVoice 	M 	train-other-500
    1820, # US	Feyaza 	F 	train-other-500
    1771, # US Chelsea S. 	F 	train-other-500
    1718, # US	Caroline Driggs 	F 	train-other-500
    1752, # US Shana Cohen 	F 	train-clean-360
    1869, # US	NastassiaS 	F 	train-other-500
    1863, # US	Tika Sabu 	F 	train-other-500
    1723, # US 	Rachel Bossier 	F 	train-other-500
    1798, # US	C. L. W. Rollins 	F 	train-other-500
    1755, # US Yvonne Smith 	F 	train-clean-360
    1738, # US	Lois C. Johnson 	F 	train-clean-360
    1887, # US 	Jenna Lanman 	F 	train-clean-360
]
# chunk 10/11
speaker_ids10_11 = [
    1956,  # Thomas Meaney
    2049,  # AdrianBisson
    1978,  # John Trevithick
    2001,  # Wesseling
    2114,  # Larry Beasley
    2032,  # doonaboon
    2087,  # James Bendall
    2011,  # pekein
    2056,  # acloward
    2007,  # Art Leung
    2084,  # Eberle Thomas
    2115,  # Pete Milan
    1987,  # Andrew White
    1959,  # DVoice
    1954,  # Szindbad
    2036,  # T.K. Kirven
    1947,  # Barbara Edelman
    2045,  # Linda Ciano
    1979,  # roeg11
    2075,  # Joy S Grape
    2091,  # Caroline Hemmerly Kunkle
    2023,  # Vickie Ranz
    2014,  # Eden Rea-Hedrick
    1965,  # redhed3095
    1989,  # Joannemmp
    2040,  # MJ Franck
    1996,  # Mary in Arkansas
    1957,  # Sarika Pawar
    2100,  # Katherine
    2069,  # Asta1234
    2096,  # Tara Dow
    2095,  # Diana Dolan
    1995,  # Jill Janovetz
    2017,  # CaprishaPage
    2010,  # Peggy
    1998,  # voicebynatalie
    1952,  # Katalina Watt
    2094,  # Meg Cowan
    2065,  # Muriel
    2312,  # Jon Kerfoot
    2217,  # Jesse Crisp-Sears
    2197,  # Mike Nelson
    2282,  # Robert Snoza
    2192,  # Sammy Bean
    2268,  # Greg Giordano
    2278,  # Jake Woldstad
    2241,  # Steven Reynolds
    2239,  # amaskill
    2225,  # nomorejeffs
    2283,  # Tim Cote
    2230,  # Sam Naishtat
    2151,  # MaxSitting
    2141,  # KateC
    2314,  # Cheri Jordan
    2127,  # Ron Lockhart
    2147,  # Shawn Bayern
    2251,  # Wiley Combs
    2195,  # Lynne Thompson
    2272,  # JamesMcAndrew
    2156,  # C F de Rosset
    2292,  # Arnold
    2143,  # Suebee
    2333,  # Anita Slusser
    2233,  # Alexis Castro
    2305,  # Brooke Cunningham
    2247,  # Lois Browne
    2171,  # Carolyne
    2172,  # Demosthenes
    2291,  # lewildesen
    2194,  # Iridescat
    2331,  # Madam Fickle
    2317,  # helengraves
    2234,  # Coreena
    2209,  # Samantha J Gubitz
    2152,  # Kristel Tretter
    2267,  # Frances Brown
    2275,  # NatalieOram
    2298,  # Sheila Wood
    2138,  # Jeannie Tirado
    2220,  # Loveday
]
# chunk_12
speaker_ids12 = [
    2403, # 	Ian Quinlan 	M 	train-clean-360
    2436, # IND	josembi 	M 	train-other-500
    2387, # 	Brett G. Hirsch 	M 	train-other-500
    2444, # 	dsilber01 	M 	train-clean-360
    2419, # Gary Dana 	M 	train-clean-100
    2453, # 	Krzysztof Rowinski 	M 	train-clean-360
    2451, #	DeanOBuchanan 	M 	train-clean-100
    2473, #	Eric Metzler 	M 	train-clean-360
    2415, #	Patrick Eaton 	M 	train-other-500
    2379, #	pjhoury 	M 	train-other-500
    2377, #	Jon Kissack 	M 	train-clean-100
    2355, # 	yeknod 	M 	train-other-500
    2452, #	Walt Allan 	M 	train-other-500
    2401, #	Matt Parker 	M 	train-clean-360
    2359, #	Doug Reed 	M 	train-other-500
    2425, #	noblesavage 	M 	train-clean-100
    2390, #	sdaeley17 	M 	train-clean-360
    2461, #	ScottReyonoldsVoice 	M 	train-clean-360
    2371, #	Alexander Hatton 	M 	train-clean-360
    2479, #	Daisy Flaim 	F 	train-clean-100
    2483, #	Tammy Porter 	F 	train-clean-360
    2372, #	Lynne Ray 	F 	train-clean-360
    2422, #	Jude Somers 	F 	train-clean-360
    2357, #	William Gavula 	M 	train-other-500
    2439, #	KHand 	F 	train-clean-360
    2441, # 	Alison Stewart 	F 	train-clean-360
    2413, # 	Joanne Rochon 	F 	train-clean-360
    2383, #	Emma Joyce 	F 	train-other-500
    2378, #	Jackie Drown 	F 	train-clean-360
    2352, #	Jaimie Noy 	F 	train-clean-100
    2397, # 	Rebecca Braunert-Plunkett 	F 	train-other-500
    2394, #	TinaNygard2 	F 	train-clean-100
    2447, #	Deena Rhoads 	F 	train-clean-360
    2358, # 	Betty Perry 	F 	train-clean-360
    2471, #	MariaS 	F 	train-other-500
    2468, #	Erin Schellhase 	F 	train-clean-360
    2370, # 	gloriousjob 	M 	train-clean-360
    2341, # 	Haili 	F 	train-other-500
    2469, #	Kevin Owens 	M 	train-clean-100
    2448, #	Emily Maynard 	F 	train-clean-360
    2351, # Nick Bulka 	M 	train-other-500
]

speaker_ids = speaker_ids1 + speaker_ids2 + speaker_ids3 + speaker_ids4 + speaker_ids5 + speaker_ids6 + speaker_ids7 + speaker_ids8 + speaker_ids9 + speaker_ids10_11 + speaker_ids12

len(speaker_ids), len(set(speaker_ids))

(599, 599)

In [16]:
from sklearn.cluster import KMeans
from scipy.spatial import distance

def gen_cluster_table(batch: list, embeddings: list, batch_idx: int, n_clusters: int = 3):
    # df = pd.DataFrame(batch)
    embeddings = np.array(embeddings) # type: ignore

    # Cluster the embeddings
    kmeans = KMeans(n_clusters=n_clusters, n_init=10).fit(embeddings)

    # Add the cluster labels to your data
    for data, label, X_s in zip(batch, kmeans.labels_, embeddings):
        data["cluster"] = label

        center = kmeans.cluster_centers_[label]
        # Calculate the distance from the cluster center
        data["distance"] = distance.euclidean(X_s, center)

    df = pd.DataFrame(batch)

    # Get the unique cluster labels
    clusters = df["cluster"].sort_values().unique()
    metrics = ["execution_time", "ermr", "jitter", "shimmer", "cluster", "distance"]

    demo_dirname = f"logs/{dataset_url}_{checkpoint_name}"
    os.makedirs(demo_dirname, exist_ok=True)

    # Add header
    result = f"""
    <h3>Chunk #{batch_idx}: </h3>
    {df[["speaker_id", *metrics]].describe().to_html()}
    <h3>Chunk #{batch_idx} audio data: </h3>
    """

    # Loop over the clusters
    for cluster in clusters:
        df_cluster = df[df["cluster"] == cluster].sort_values(by=["distance"], ascending=True)

        # Add header
        result = f"""
        <h3>Cluster #{cluster}: </h3>
        """

        # Generate and display the table for the chunk
        # for _, row in df_ermr_sorted.iterrows():
        result += gen_table(df_cluster.to_dict("records")).data # type: ignore

        # Save result as HTML
        with open(f"{demo_dirname}/chunk_{batch_idx}.html", "w") as f:
            f.write(result)


In [17]:
from resemblyzer import VoiceEncoder, preprocess_wav


# text_tts = """
# Casablanca: “But what about us?”
# “We’ll always have Paris.”
# The Wizard of Oz: “Lions? And Tigers? And Bears?”
# “Oh my!”
# Star Wars (A New Hope): “He’s almost in range.”
# “That’s no moon; it’s a space station.”
# Love Story: “Jenny, I’m sorry.”
# “Don’t. Love means never having to say you’re sorry.”
# No Country for Old Men: “Look, I need to know what I stand to win.”
# “Everything.”
# Forrest Gump: “I thought I’d try out my sea legs.”
# “But you ain’t got no legs, Lieutenant Dan.”
# Toy Story: “Buzz, you’re flying!”
# “This isn’t flying; this is falling with style.”
# As the snake shook its head, a deafening shout behind Harry made both of them jump.
# ‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
# """

text_tts = """
The Wizard of Oz: “Lions? And Tigers? And Bears?”
“Oh my!”
"""

with open("training/datasets/speaker_id_mapping_libri.json") as f:
    id_mapping = json.load(f)

batch_size = 10
model.vocoder_module.to(device)

metrics = Metrics()
text_demo_with_metrics = []

# Initialize the voice encoder
encoder = VoiceEncoder()
# Prepare an array to hold the embeddings
embeddings = []

speakers_df = pd.read_csv(
    "./datasets_cache/LIBRITTS/LibriTTS/speakers.tsv",
    sep="\t",
    names=["READER", "GENDER", "SUBSET", "NAME"],
)

subsets = ["train-clean-100", "train-clean-360", "train-other-500"]

speakers = speakers_df[speakers_df['SUBSET'].isin(subsets)]

start_chunk = 0  # The chunk to start from
start_idx = start_chunk * batch_size  # The index to start from

speaker_ids_set = set(speaker_ids)

real_idx = 0
for idx, speaker in enumerate(speakers.to_dict("records")):
    # Skip the speakers that have already been processed
    if idx <= start_idx:
        continue

    speaker_id = speaker["READER"]
    speaker_id = id_mapping.get(str(speaker_id))

    if speaker_id is None or speaker_id not in speaker_ids_set:
        continue

    speaker_name = speaker["NAME"]
    gender = speaker["GENDER"]
    subset = speaker["SUBSET"]

    with torch.no_grad():
        speaker_id_ = torch.tensor([int(speaker_id)], device=device)
        start_time = time.time()
        wav = model.forward(text_tts, speaker_id_)
        end_time = time.time()
        (
            ermr,
            jitter,
            shimmer,
        ) = metrics.wav_metrics(wav.unsqueeze(0))
        execution_time = round(end_time - start_time, 2)

        mel = tacotronSTFT.get_mel_from_wav(wav)

        # Generate an embedding for the speaker
        embed = encoder.embed_utterance(wav.detach().cpu().numpy())
        # Append the embedding to our list of embeddings
        embeddings.append(embed)

        text_demo_with_metrics.append({
            "speaker_id": speaker_id,
            "speaker_name": speaker_name,
            "gender": gender,
            "subset": subset,
            "wav": wav.detach().cpu().numpy(),
            "execution_time": execution_time,
            "mel": mel.detach().cpu().numpy(),
            "ermr": ermr,
            "jitter": jitter,
            "shimmer": shimmer,
        })

    if real_idx > 0 and real_idx % batch_size == 0:
        gen_cluster_table(text_demo_with_metrics, embeddings, real_idx // batch_size)
        text_demo_with_metrics = []
        break

    real_idx += 1
    torch.cuda.empty_cache()

gen_cluster_table(text_demo_with_metrics, embeddings, 999)


Loaded the voice encoder model on cuda in 0.02 seconds.
       speaker_id  execution_time       ermr     jitter    shimmer    cluster  \
count   11.000000       11.000000  11.000000  11.000000  11.000000  11.000000   
mean    26.454545        0.097273  12.196014   0.028277   0.000882   0.909091   
std     12.266733        0.015551   6.033388   0.032268   0.000554   0.943880   
min      8.000000        0.090000   3.170208  -0.016780   0.000061   0.000000   
25%     16.500000        0.090000   7.291003   0.002589   0.000503   0.000000   
50%     30.000000        0.090000  12.324289   0.025506   0.000795   1.000000   
75%     37.000000        0.095000  17.704631   0.055425   0.001310   2.000000   
max     42.000000        0.140000  20.255714   0.081237   0.001891   2.000000   

        distance  
count  11.000000  
mean    0.279373  
std     0.043543  
min     0.223680  
25%     0.242515  
50%     0.291456  
75%     0.306329  
max     0.354393  
clusters [0 1 2]
       speaker_id  executi

ValueError: Cannot describe a DataFrame without columns

In [ ]:
import os
import shutil
import torchaudio

# TODO: in progress

# Define the speaker IDs to keep
speaker_ids_to_keep = speaker_ids5  # Replace with your actual speaker IDs

# Define the maximum audio length in seconds
max_audio_length = 30

# Define the source and destination directories
source_dir = '/path/to/libritts'  # Replace with your actual LibriTTS directory
dest_dir = '/path/to/libri-selected'  # Replace with your actual destination directory

# Iterate over the subsets in the source directory
for subset in os.listdir(source_dir):
    subset_dir = os.path.join(source_dir, subset)

    # Skip if not a directory
    if not os.path.isdir(subset_dir):
        continue
    
    # Iterate over the speakers in the subset
    for speaker in os.listdir(subset_dir):
        speaker_id = int(speaker)
        
        # Skip if the speaker is not in the list of speaker IDs to keep
        if speaker_id not in speaker_ids_to_keep:
            continue
        
        speaker_dir = os.path.join(subset_dir, speaker)
        
        # Iterate over the chapters in the speaker directory
        for chapter in os.listdir(speaker_dir):
            chapter_dir = os.path.join(speaker_dir, chapter)
            
            # Iterate over the files in the chapter directory
            for file in os.listdir(chapter_dir):
                if file.endswith('.wav'):
                    file_path = os.path.join(chapter_dir, file)
                    
                    # Load the audio file
                    waveform, sample_rate = torchaudio.load(file_path)
                    
                    # Calculate the audio length in seconds
                    audio_length = waveform.shape[1] / sample_rate
                    
                    # If the audio length is less than or equal to the maximum, copy it to the destination directory
                    if audio_length <= max_audio_length:
                        dest_file_path = file_path.replace(source_dir, dest_dir)
                        dest_file_dir = os.path.dirname(dest_file_path)
                        
                        # Create the destination directory if it doesn't exist
                        os.makedirs(dest_file_dir, exist_ok=True)
                        
                        # Copy the file
                        shutil.copyfile(file_path, dest_file_path)
                        
                        # Copy the associated .normalized.txt and .original.txt files
                        for ext in ['.normalized.txt', '.original.txt']:
                            txt_file_path = file_path.replace('.wav', ext)
                            if os.path.exists(txt_file_path):
                                dest_txt_file_path = txt_file_path.replace(source_dir, dest_dir)
                                shutil.copyfile(txt_file_path, dest_txt_file_path)